In [1]:
from stable_baselines3.common.callbacks import BaseCallback
import numpy as np

class RewardAndGoalLoggingCallback(BaseCallback):
    def __init__(self, log_interval=20000, verbose=0):
        super().__init__(verbose)
        self.log_interval = log_interval
        self.cumulative_reward = 0
        self.step_counter = 0

    def _on_step(self):
        reward = self.locals["rewards"][0]
        self.cumulative_reward += reward
        self.step_counter += 1

        if self.step_counter % self.log_interval == 0:
            # Get the goal_pos from the environment (VecEnv-safe)
            goal_pos = self.training_env.get_attr("goal_pos")[0]
            ef_pos = self.training_env.get_attr("robot")[0].get_joint_obs()["ee_pos"]
            print(f"📈 Step {self.num_timesteps} | Reward (last {self.log_interval}): {round(self.cumulative_reward, 4)} | 🎯 Goal: {np.round(goal_pos, 3)},f🤖 EE Pos: {np.round(ef_pos, 3)} ")
            self.cumulative_reward = 0

        return True


In [ ]:
from stable_baselines3 import PPO
from env import UArmEnv

# Create environment and model
env = UArmEnv(render=False)
model = PPO(
    "MlpPolicy",
    env,
    ent_coef=0.005,        # Encourages broader exploration without overwhelming reward
    n_steps=2048,          # Longer rollout → better credit assignment
    batch_size=256,        # Good match for long rollouts
    gae_lambda=0.95,       # Slightly longer-term reward tracking
    gamma=0.99,            # Standard long-term discount
    verbose=0
)

# Add logging callback
reward_logger = RewardAndGoalLoggingCallback(log_interval=20000)

# Train with logging
model.learn(total_timesteps=1_000_000, callback=reward_logger)

env.close()

model.save("ppo_uarm_pos_final_30")

pybullet build time: Jan 29 2025 23:16:28


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/fatduck/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


⏱️ Timeout at Step 500 | EE: [ 0.061 -0.06   0.34 ] | Goal: [ 0.037 -0.32   0.025]
⏱️ Timeout at Step 500 | EE: [ 0.025 -0.231  0.121] | Goal: [ 0.114 -0.322  0.06 ]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 340      |
|    ep_rew_mean     | -84.3    |
| time/              |          |
|    fps             | 459      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
⏱️ Timeout at Step 500 | EE: [-0.026 -0.286  0.19 ] | Goal: [ 0.112 -0.317  0.025]
⏱️ Timeout at Step 500 | EE: [ 0.022 -0.088  0.292] | Goal: [ 0.036 -0.318  0.06 ]
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 339         |
|    ep_rew_mean          | -80.7       |
| time/                   |             |
|    fps                  | 427         |
|    iterations           | 2           |
|    time_elapsed         | 9     

In [ ]:
model.save("ppo_uarm")
1 000 000